In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
import torch.optim.lr_scheduler as scheduler
from torch.utils.data import random_split,DataLoader

import torchvision
from torchvision.datasets import CIFAR10,CIFAR100
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor,Normalize

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device(0))

cuda
0
1
GeForce RTX 3090


In [23]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [24]:
ds10=CIFAR10(root="data",transform=transform)
ts10=CIFAR10(root="data",train=False,transform=transform)

In [25]:
tr10,vs10,_=random_split(ds10,[10000,10000,30000],torch.manual_seed(1))

In [26]:
class_count={}
for i in tr10:
    if i[1] not in class_count:
        class_count[i[1]]=0
    class_count[i[1]]+=1
print(class_count)

{0: 1010, 1: 1011, 6: 978, 3: 1002, 4: 1011, 8: 1048, 7: 998, 2: 999, 5: 982, 9: 961}


In [27]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(784,784*4),
            nn.ReLU(),
            nn.Linear(784*4,784*12),
            nn.ReLU(),
            nn.Linear(784*12,784*24),
            nn.ReLU(),
            nn.Linear(784*24,784*24),
            nn.ReLU(),
            nn.Linear(784*24,784*12),
            nn.ReLU(),
            nn.Linear(784*12,784*4),
            nn.ReLU(),
            nn.Linear(784*4, 784),
            nn.ReLU(),
            nn.Linear(784, 120),
            nn.ReLU(),
            nn.Linear(120, 30),
            nn.ReLU(),
            nn.Linear(30,10))
        
        def forward(self, X):
            X=
            return self.layer(X)
            

In [28]:
mlp=MLP().to(device)

In [29]:
learning_rate=5e-4
training_epochs=30
batch_size=50

In [30]:
obj=nn.CrossEntropyLoss().to(device)
opt=Adam(mlp.parameters(),lr=0.001)

In [31]:
tr10_loader=DataLoader(dataset=tr10,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True)

vs10_loader=DataLoader(dataset=tr10,
                      batch_size=batch_size,
                      shuffle=False)

In [32]:
training_loss_history=[]
validation_loss_history=[]

for epoch in range(training_epochs):
    training_loss=0.0
    validation_loss=0.0
    for X,y in tr10_loader:
        X=X.to(device)
        y=y.to(device)
        
        opt.zero_grad()
        output=mlp(X)
        cost=obj(output,y)
        cost.backward()
        opt.step()
        
        training_loss+=cost.item()/batch_size
        
    with torch.no_grad():
        for X,y in vs10_loader:
            X=X.to(device)
            y=y.to(device)
            
            output=mlp(X)
            cost=obj(output,y)
            
            validation_loss+=cost.item()/batch_size
                    
    training_loss_history.append(training_loss)
    validation_loss_history.append(validation_loss)
    print("[ Epoch: ",epoch," loss :",training_loss," validation :",validation_loss,"]")

NotImplementedError: 